# 허깅페이스 임베딩(HuggingFace Embeddings)

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH08-Embeddings")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH08-Embeddings


In [4]:
import os
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "토큰입력"

## 샘플 데이터

In [5]:
texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

**참고(Reference)**

![](./images/top-ranked-embeddings.png)

- [(출처) Kor-IR: 한국어 검색을 위한 임베딩 벤치마크](https://github.com/teddylee777/Kor-IR?tab=readme-ov-file)

## HuggingFace Endpoint Embedding

`HuggingFaceEndpointEmbeddings` 는 내부적으로 InferenceClient를 사용하여 임베딩을 계산한다는 점에서 HuggingFaceEndpoint가 LLM에서 수행하는 것과 매우 유사합니다. 

In [18]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

Document 임베딩은 `embed_documents()` 를 호출하여 생성할 수 있습니다.

In [19]:
%%time
# Document Embedding 수행
embedded_documents = hf_embeddings.embed_documents(texts)

HfHubHTTPError: 404 Client Error: Not Found for url: https://router.huggingface.co/hf-inference/pipeline/feature-extraction/sentence-transformers/all-mpnet-base-v2 (Request ID: Root=1-681a5c05-0f21e5f2459ad6c25cbbec75;f7443f05-cda5-47da-9ff0-368384a4c684)

In [ ]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

In [ ]:
# Document Embedding 수행
embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")

In [ ]:
len(embedded_query)

### 유사도 계산

**벡터 내적을 통한 유사도 계산**
- 벡터 내적(dot product)을 사용하여 유사도를 계산합니다. 

- 유사도 계산 공식:

$$ \text{similarities} = \mathbf{query} \cdot \mathbf{documents}^T $$

#### 벡터 내적의 수학적 의미

**벡터 내적 정의**

벡터 $\mathbf{a}$와 $\mathbf{b}$의 내적은 다음과 같이 정의됩니다:
$$ \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i $$

**코사인 유사도와의 관계**

벡터 내적은 다음과 같은 성질을 가집니다.
$$ \mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos \theta $$

여기서,
- $\|\mathbf{a}\|$와 $\|\mathbf{b}\|$는 각각 벡터 $\mathbf{a}$와 $\mathbf{b}$의 크기(노름, Euclidean norm)입니다.
- $\theta$는 두 벡터 사이의 각도입니다.
- $\cos \theta$는 두 벡터 사이의 코사인 유사도입니다.

**벡터 내적의 유사도 해석**
내적 값이 클수록 (양의 큰 값일수록),
- 두 벡터의 크기($\|\mathbf{a}\|$와 $\|\mathbf{b}\|$)가 크고,
- 두 벡터 사이의 각도($\theta$)가 작으며 ($\cos \theta$가 1에 가까움),

이는 두 벡터가 유사한 방향을 가리키고, 크기가 클수록 더 유사하다는 것을 의미합니다.

**벡터의 크기(노름) 계산**

Euclidean norm 정의
벡터 $\mathbf{a} = [a_1, a_2, \ldots, a_n]$에 대해, Euclidean norm $\|\mathbf{a}\|$는 다음과 같이 정의됩니다:
$$ \|\mathbf{a}\| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} $$

----

query 와 embedding_document 간의 유사도 계산

In [ ]:
import numpy as np

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

In [ ]:
sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]
sorted_idx

In [ ]:
print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

## HuggingFace Embeddings



### `intfloat/multilingual-e5-large-instruct`

- [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct)
- [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large)

In [25]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt", encoding="UTF-8").load()

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 임베딩 모델 설정
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)

# 문서로부터 FAISS 인덱스 생성 및 검색기 설정
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 10}
)

# 질의 설정
query = "Word2Vec 에 대해서 알려줄래?"

# 질의 수행 및 결과 문서 반환
docs = retriever.invoke(query)

In [26]:
docs

[Document(id='2369aa4f-ffa8-485c-9535-adb8309e5da4', metadata={'source': './data/appendix-keywords.txt'}, page_content='Open Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser'),
 Document(id='9c267abb-13d4-4c31-a31f-bd513a3580fa', metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Sou

In [20]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/msmarco-distilbert-dot-v5"
# model_name = "intfloat/multilingual-e5-large"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.27k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
%time
# Document
embedded_documents1 = hf_embeddings.embed_documents(texts)

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.05 μs


In [23]:
embedded_documents1

[[-0.020188499242067337,
  -0.013132541440427303,
  0.01254001259803772,
  -0.030314741656184196,
  0.003673213766887784,
  0.04381757974624634,
  0.01450064592063427,
  0.0029913221951574087,
  -0.02164742536842823,
  -0.006854040548205376,
  0.002446005353704095,
  0.031712863594293594,
  -0.01350339874625206,
  0.0045795738697052,
  0.002590346848592162,
  0.030603425577282906,
  0.01779695227742195,
  0.006009725388139486,
  -0.0384242869913578,
  0.021939989179372787,
  0.006877100095152855,
  0.005745681934058666,
  0.06233561784029007,
  0.03389092907309532,
  0.03352740779519081,
  -0.010539996437728405,
  -0.03644363582134247,
  -0.02557951770722866,
  0.029688986018300056,
  0.013014041818678379,
  0.025928376242518425,
  -0.018473703414201736,
  0.0660453587770462,
  -0.010092342272400856,
  0.02942965179681778,
  0.0068241930566728115,
  0.005572532769292593,
  0.024548541754484177,
  0.015562828630208969,
  0.023853600025177002,
  -0.039844732731580734,
  0.000208701574592

In [22]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		sentence-transformers/msmarco-distilbert-dot-v5


NameError: name 'embedded_documents' is not defined

## BGE-M3 임베딩

아래의 옵션 중 에러가 발생할 수 있는 옵션 설명

- `{"device": "mps"}`: GPU 대신 MPS를 사용하여 임베딩 계산을 수행합니다. (Mac 사용자)
- `{"device": "cuda"}`: GPU를 사용하여 임베딩 계산을 수행합니다. (Linux, Windows 사용자, 단 CUDA 설치 필요)
- `{"device": "cpu"}`: CPU를 사용하여 임베딩 계산을 수행합니다. (모든 사용자

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "mps"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [ ]:
%time
# Document
embedded_documents = hf_embeddings.embed_documents(texts)

In [ ]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

In [ ]:
import numpy as np

embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")
embedded_documents = hf_embeddings.embed_documents(texts)

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

### `FlagEmbedding` 을 활용하는 방식

**참고**
- [FlagEmbedding - BGE-M3 Usage](https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/BGE_M3#usage)


`FlagEmbedding` 에서 제공하는 세 가지 접근법을 조합하면, 더욱 강력한 검색 시스템을 구축할 수 있습니다.

- Dense Vector: BGE-M3의 다국어, 다중 작업 능력을 기반으로 함
- Lexical weight를 활용한 sparse embedding으로 정확한 단어 매칭을 수행
- ColBERT의 multi-vector 접근법으로 문맥을 고려한 세밀한 매칭 수행

In [ ]:
# FlagEmbedding 설치
!pip install -qU FlagEmbedding, torch, torchvision

In [1]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"
bge_embeddings = BGEM3FlagModel(
    model_name, use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.

bge_embedded = bge_embeddings.encode(
    texts,
    batch_size=12,
    max_length=8192,  # 이렇게 긴 길이가 필요하지 않은 경우 더 작은 값을 설정하여 인코딩 프로세스의 속도를 높일 수 있습니다.
)["dense_vecs"]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

model.onnx_data:  34%|###4      | 776M/2.27G [00:00<?, ?B/s]

pytorch_model.bin:  33%|###2      | 744M/2.27G [00:00<?, ?B/s]

NameError: name 'texts' is not defined

In [ ]:
import platform

print(platform.platform())

import torch

print(torch.__version__)

torch.backends.mps.is_available()

In [ ]:
max_memory = torch.mps.recommended_max_memory()
print("Recommended Max Memory : ", (max_memory / (1024 * 1024 * 1024)), "GB")

In [ ]:
bge_embedded.shape

In [ ]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

In [ ]:
from FlagEmbedding import BGEM3FlagModel

bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_dense=True)

In [ ]:
# 결과 출력(행, 열)
bge_encoded["dense_vecs"].shape

### Sparse Embedding (Lexical Weight)

Sparse embedding은 벡터의 대부분의 값이 0인 고차원 벡터를 사용하는 임베딩 방식입니다. Lexical weight를 활용한 방식은 단어의 중요도를 고려하여 임베딩을 생성합니다.

**작동 방식**
1. 각 단어에 대해 lexical weight(어휘적 가중치)를 계산합니다. 이는 TF-IDF나 BM25 같은 방법을 사용할 수 있습니다.
2. 문서나 쿼리의 각 단어에 대해, 해당 단어의 lexical weight를 사용하여 sparse vector의 해당 차원에 값을 할당합니다.
3. 결과적으로 문서나 쿼리는 대부분의 값이 0인 고차원 벡터로 표현됩니다.

**장점**
- 단어의 중요도를 직접적으로 반영할 수 있습니다.
- 특정 단어나 구문을 정확히 매칭할 수 있습니다.
- 계산이 상대적으로 빠릅니다.

In [ ]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_sparse=True)

In [ ]:
lexical_scores1 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][0]
)
lexical_scores2 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][1]
)
# 0 <-> 0
print(lexical_scores1)
# 0 <-> 1
print(lexical_scores2)

### Multi-Vector (ColBERT)

ColBERT(Contextualized Late Interaction over BERT)는 문서 검색을 위한 효율적인 방법입니다. 이 방식은 문서와 쿼리를 여러 개의 벡터로 표현하는 multi-vector 접근법을 사용합니다.

**작동 방식**

1. 문서의 각 토큰에 대해 별도의 벡터를 생성합니다. 즉, 하나의 문서는 여러 개의 벡터로 표현됩니다.
2. 쿼리도 마찬가지로 각 토큰에 대해 별도의 벡터를 생성합니다.
3. 검색 시, 쿼리의 각 토큰 벡터와 문서의 모든 토큰 벡터 사이의 유사도를 계산합니다.
4. 이 유사도들을 종합하여 최종 검색 점수를 계산합니다.

**장점**
- 토큰 수준의 세밀한 매칭이 가능합니다.
- 문맥을 고려한 임베딩을 생성할 수 있습니다.
- 긴 문서에 대해서도 효과적으로 작동합니다.

In [ ]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_colbert_vecs=True)

In [ ]:
colbert_scores1 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][0]
)
colbert_scores2 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][1]
)
# 0 <-> 0
print(colbert_scores1)
# 0 <-> 1
print(colbert_scores2)